<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 4 Sprint 3 Assignment 1*

# Recurrent Neural Networks and Long Short Term Memory (LSTM)

![Monkey at a typewriter](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3c/Chimpanzee_seated_at_typewriter.jpg/603px-Chimpanzee_seated_at_typewriter.jpg)

It is said that [infinite monkeys typing for an infinite amount of time](https://en.wikipedia.org/wiki/Infinite_monkey_theorem) will eventually type, among other things, the complete works of Wiliam Shakespeare. Let's see if we can get there a bit faster, with the power of Recurrent Neural Networks and LSTM.

This text file contains the complete works of Shakespeare: https://www.gutenberg.org/files/100/100-0.txt

Use it as training data for an RNN - you can keep it simple and train character level, and that is suggested as an initial approach.

Then, use that trained RNN to generate Shakespearean-ish text. Your goal - a function that can take, as an argument, the size of text (e.g. number of characters or lines) to generate, and returns generated text of that size.

Note - Shakespeare wrote an awful lot. It's OK, especially initially, to sample/use smaller data and parameters, so you can have a tighter feedback loop when you're trying to get things running. Then, once you've got a proof of concept - start pushing it more!

In [1]:
# imports

from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.optimizers import RMSprop

import numpy as np
import random
import re

import sys

In [2]:
# Loading in some Borges. Shakespeare is boring!

with open(f'./borges/borges_collected_fictions.txt', 'r', encoding='utf8') as f:
            data = f.read()

In [3]:
print(data[:500])

PENGUIN BOOKS





COLLECTED FICTIONS





Jorge Luis Borges was born in Buenos Aires in 1899 and was educated in Europe. One of the most widely acclaimed writers of our time, he published many collections of poems, essays, and short stories before his death in Geneva in June 1986. In 1961 Borges shared the International Publishers’ prize with Samuel Beckett. The Ingram Merrill Foundation granted him its Annual Literary Award in 1966 for his “outstanding contribution to literature.” In 1971 Colu


In [4]:
def cleaner(text):
    '''cleans the text of escape chars and the like.'''
    stripped = text.strip(' ')
    lowered = stripped.lower()
    cleaned = re.sub(r"[^a-zA-Z ^0-9 \. , ']", ' ', lowered)
    despaced = cleaned.strip(' ')
    return despaced

In [5]:
# clean the data
clean_data = cleaner(data)

In [6]:
# check that things look alright
clean_data[:400]

'penguin books      collected fictions      jorge luis borges was born in buenos aires in 1899 and was educated in europe. one of the most widely acclaimed writers of our time, he published many collections of poems, essays, and short stories before his death in geneva in june 1986. in 1961 borges shared the international publishers  prize with samuel beckett. the ingram merrill foundation granted '

In [7]:
# make a list of all unique words, check how many
wordlist = clean_data.split(' ')
wordset = list(set(clean_data.split(' ')))
len(wordset)

26646

In [8]:
# get rid of words below a certain frequency
freqdict = {word:wordlist.count(word) for word in wordset}

In [10]:
freqdict['i']

3723

In [11]:
smaller_list = [ word for word in wordlist if (freqdict[word] >= 10) and (word != '') ] 

In [12]:
len(set(smaller_list))

2246

In [13]:
small_wordset = list(set(smaller_list))

In [14]:
# encode words as ints

words_to_ints = {w:i for i,w in enumerate(small_wordset)}
ints_to_words = {i:w for i,w in enumerate(small_wordset)}

In [15]:
words_to_ints['and']

1502

In [16]:
# Create the Sequence Data

maxlen = 40
step = 5

encoded = [words_to_ints[w] for w in smaller_list]

sequences = []
next_words = []

for ii in range(0, len(encoded)-maxlen, step):
    sequences.append(encoded[ii : ii+maxlen])
    next_words.append(encoded[ii+maxlen])

In [17]:
len(smaller_list)

172530

In [18]:
# Specify x & y

x = np.zeros((len(sequences), maxlen, len(small_wordset)), dtype=np.bool)
y = np.zeros((len(sequences), len(small_wordset)), dtype=np.bool)

for ii, sequence in enumerate(sequences):
    for t, word in enumerate(sequence):
        x[ii,t,word] = 1
        
    y[ii, next_words[ii]] = 1

In [19]:
y.shape

(34498, 2246)

In [ ]:
# build the model: a single LSTM

model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(small_wordset))))
model.add(Dense(len(small_wordset), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

In [443]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [444]:
def generate_text(seed, num_words, diversity):
    clean_seed = cleaner(seed)
    seed_list = clean_seed.split(' ')
    
    maxlen = 40
    step = 1

    encoded = [words_to_ints[w] for w in seed_list]
    to_pad = 40 - len(encoded)
    
    if len(encoded) < 40:
        zeros = [0] * to_pad
        sequence = zeros + encoded
    elif len(encoded) > 40:
        sequence = encoded[-40:]
    else:
        sequence = encoded
        

    for ii in range(0,num_words):
        
        sequence_last40 = sequence[-40:]
        
        x_pred = np.zeros((1, maxlen, len(small_wordset)))
        for t, word in enumerate(sequence_last40):
            x_pred[0, t, word] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_word = sample(preds, diversity)

        sequence.append(next_word)
        
    output = [ints_to_words[w] for w in sequence][to_pad:]
    str_out = ' '.join(output)
    return str_out

In [445]:
def on_epoch_end(epoch, _):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(smaller_list) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('----- diversity:', diversity)

        
        input_list = smaller_list[start_index: start_index + maxlen]
        input_sentence = ' '.join(input_list)
        print('----- Generating with seed: "' + input_sentence + '"')
        output = generate_text(input_sentence, 20, diversity)

            
        print(output)

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [446]:
with tf.device('/gpu:0'):
    model.fit(x, y,
              batch_size=128,
              epochs=20,
              callbacks=[print_callback])

Train on 34498 samples
Epoch 1/20
34432/34498 [============================>.] - ETA: 0s - loss: 5.9778
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "i do and you do not in others still, we both do. in this one, which the hand of chance has me, you have come to my home in another, when you come through my garden you find me dead"
i do and you do not in others still, we both do. in this one, which the hand of chance has me, you have come to my home in another, when you come through my garden you find me dead the the of the the the in the the the the the the the the the the the the of
----- diversity: 0.5
----- Generating with seed: "i do and you do not in others still, we both do. in this one, which the hand of chance has me, you have come to my home in another, when you come through my garden you find me dead"
i do and you do not in others still, we both do. in this one, which the hand of chance has me, you have come to my home in another, when 

In [457]:
generate_text('the knife of the gaucho lay in the sand',50, 0.8)

"the knife of the gaucho lay in the sand the city of this with the which is su to the story of the the names world was sometimes in his he was long of the house man's i was a dark and and the only two the of the which is taking of his and the published one of"

# Resources and Stretch Goals

## Stretch goals:
- Refine the training and generation of text to be able to ask for different genres/styles of Shakespearean text (e.g. plays versus sonnets)
- Train a classification model that takes text and returns which work of Shakespeare it is most likely to be from
- Make it more performant! Many possible routes here - lean on Keras, optimize the code, and/or use more resources (AWS, etc.)
- Revisit the news example from class, and improve it - use categories or tags to refine the model/generation, or train a news classifier
- Run on bigger, better data

## Resources:
- [The Unreasonable Effectiveness of Recurrent Neural Networks](https://karpathy.github.io/2015/05/21/rnn-effectiveness/) - a seminal writeup demonstrating a simple but effective character-level NLP RNN
- [Simple NumPy implementation of RNN](https://github.com/JY-Yoon/RNN-Implementation-using-NumPy/blob/master/RNN%20Implementation%20using%20NumPy.ipynb) - Python 3 version of the code from "Unreasonable Effectiveness"
- [TensorFlow RNN Tutorial](https://github.com/tensorflow/models/tree/master/tutorials/rnn) - code for training a RNN on the Penn Tree Bank language dataset
- [4 part tutorial on RNN](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) - relates RNN to the vanishing gradient problem, and provides example implementation
- [RNN training tips and tricks](https://github.com/karpathy/char-rnn#tips-and-tricks) - some rules of thumb for parameterizing and training your RNN